In [18]:
!pip install -q yfinance
!pip install -q matplotlib
!pip install -q pandas_datareader
!pip install -q lightgbm
!pip install scikit-learn
!pip install lightgbm --upgrade
!pip install graphviz




[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\theaw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\theaw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\theaw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import numpy as np
import datetime
import lightgbm
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import graphviz
from sklearn.preprocessing import LabelEncoder





# Economy Sample Stock

In [25]:
# Get the current stock price, pct change in price, and previous pct change in price
def stock_info(macro_indicators):
    ticker_data_info = pd.DataFrame()
    tick = yf.download(ticker, start = start_date_with_shift, end = end_date)
    ticker_data_info['average_stock_price'] = tick['Close'].resample(period_length).mean()
    ticker_data_info['average_stock_price_pct_change'] = (ticker_data_info['average_stock_price'] / ticker_data_info['average_stock_price'].shift(1) - 1) * 100
    ticker_data_info['lag_average_stock_price_pct_change'] = ticker_data_info['average_stock_price_pct_change'].shift(1)
    ticker_data_info.reset_index(inplace=True)
    macro_indicators = pd.merge(macro_indicators, ticker_data_info[['Date', 'average_stock_price', 'average_stock_price_pct_change', 'lag_average_stock_price_pct_change']], 
                                left_on = 'DATE',
                                right_on = 'Date',
                                how = 'left')
    macro_indicators.drop(columns=['Date'], inplace=True)
    
    return macro_indicators


# Money Supply

In [4]:
def fetch_money_supply():
    # Define the list of money supply series to fetch
    money_supply_series = ['M1SL', 'M2SL']

    # Fetch money supply data using pandas_datareader
    money_supply_data = pdr.get_data_fred(money_supply_series, start_date_with_shift, end_date)

    # Calculate the YoY percentage change
    money_supply_data['M1SL_YoY'] = money_supply_data['M1SL'].pct_change(12) * 100
    money_supply_data['M2SL_YoY'] = money_supply_data['M2SL'].pct_change(12) * 100


    return money_supply_data


In [5]:
# Retrieves all MS between start and end date and returns MS differences in each period bin
def money_supply_info(macro_indicators):
    money_supply_data = fetch_money_supply()
    
    money_supply_data_average = money_supply_data.resample(period_length).mean()

    money_supply_data_average['M1-M2'] = money_supply_data_average['M1SL_YoY'] - money_supply_data_average['M2SL_YoY']

    macro_indicators = pd.merge(macro_indicators, money_supply_data_average['M1-M2'], on='DATE', how='left')
    return macro_indicators

# CPI Inflation

In [6]:
def fetch_cpi_annual_change():
    try:
        # Fetch U.S. Consumer Price Index (CPI) data from FRED
        cpi_data = pdr.get_data_fred('CPIAUCSL', start = start_date_with_shift, end = end_date)

        if not cpi_data.empty:
            # Calculate annual percentage change
            cpi_data['Annual_Change'] = cpi_data['CPIAUCSL'].pct_change(12) * 100

            return cpi_data[['Annual_Change']]
        else:
            return None  # Return None if there is no data
    except Exception as e:
        return None  # Return None if there is an error


In [7]:
# Get the current CPI,  change in CPI, and previous CPI change

def cpi_info(macro_indicators):
    cpi_data_info = pd.DataFrame()
    cpi_data = fetch_cpi_annual_change()
    cpi_data_info['average_cpi_change'] = cpi_data.resample(period_length).mean()
    cpi_data_info['single_average_cpi_change'] = cpi_data_info['average_cpi_change'] - cpi_data_info['average_cpi_change'].shift(1)
    cpi_data_info['lag_single_average_cpi_change'] = cpi_data_info['single_average_cpi_change'].shift(1)

    macro_indicators = pd.merge(macro_indicators, cpi_data_info[['average_cpi_change', 'single_average_cpi_change', 'lag_single_average_cpi_change']], on = 'DATE', how = 'left')
    
    return(macro_indicators)

# Unemployment

In [8]:
def fetch_unemployment_data():
    try:
        # Define the ticker symbol for the U.S. unemployment rate (UNRATE)
        unemployment_ticker = "UNRATE"

        # Fetch unemployment data from FRED
        unemployment_data = pdr.get_data_fred(unemployment_ticker, start = start_date_with_shift, end = end_date)

        return unemployment_data
    except Exception as e:
        return None  # Return None if there is an error

In [9]:
def unemployment_info(macro_indicators):
    unemployment_info = pd.DataFrame()
    unemployment_data = fetch_unemployment_data()
    unemployment_info['average_unemployment'] = unemployment_data.resample(period_length).mean()
    unemployment_info['average_unemployment_change'] = unemployment_info['average_unemployment'] - unemployment_info['average_unemployment'].shift(1)
    unemployment_info['lag_average_unemployment_change'] = unemployment_info['average_unemployment_change'].shift(1)
    macro_indicators = pd.merge(macro_indicators, unemployment_info[['average_unemployment', 'average_unemployment_change', 'lag_average_unemployment_change']], on = 'DATE', how = 'left')
    return macro_indicators

# Interest Rates

In [10]:
def fetch_fed_interest_rate():
    try:
        # Fetch Federal Reserve interest rate data (FEDFUNDS) from FRED
        fed_interest_rate = pdr.get_data_fred('FEDFUNDS', start = start_date_with_shift, end=end_date)

        if not fed_interest_rate.empty:
            return fed_interest_rate
        else:
            return None  # Return None if there is no data
    except Exception as e:
        return None  # Return None if there is an error



In [11]:
# Get the current interest rate, interest rate change, and previous interest rate change in price
def fed_interest_rate_info(macro_indicators):
    fed_interest_rate_info = pd.DataFrame()
    fed_interest_rate = fetch_fed_interest_rate()
    fed_interest_rate_info['average_interest_rate'] = fed_interest_rate.resample(period_length).mean()
    fed_interest_rate_info['single_rate_period_change'] = fed_interest_rate_info['average_interest_rate'] - fed_interest_rate_info['average_interest_rate'].shift(1)
    fed_interest_rate_info['lag_single_rate_change'] = fed_interest_rate_info['single_rate_period_change'].shift(1)
    macro_indicators = pd.merge(macro_indicators, fed_interest_rate_info[['average_interest_rate','single_rate_period_change', 'lag_single_rate_change']],on = 'DATE', how = 'left')
    return(macro_indicators)



# CPI, Inflation, unemployment, interest rates


In [12]:
# New Function that takes the df(macro_indicators), function name, type of indicator
# Not currently used in my code yet but more efficent coding :)
def calculate_indicator_info(macro_indicators, fetch_function, column_prefix):
    indicator_info = pd.DataFrame()
    indicator_data = fetch_function()
    average_column_name = f'average_{column_prefix}'
    single_change_column_name = f'single_{column_prefix}_change'
    lag_single_change_column_name = f'lag_{single_change_column_name}'

    indicator_info[average_column_name] = indicator_data.resample(period_length).mean()
    indicator_info[single_change_column_name] = indicator_info[average_column_name] - indicator_info[average_column_name].shift(1)
    indicator_info[lag_single_change_column_name] = indicator_info[single_change_column_name].shift(1)

    macro_indicators = pd.merge(
        macro_indicators,
        indicator_info[[average_column_name, single_change_column_name, lag_single_change_column_name]],
        on='DATE',
        how='left'
    )

    return macro_indicators

In [13]:
def apply_indicators_info(macro_indicators, indicator_functions):
    for indicator_function in indicator_functions:
        macro_indicators = indicator_function(macro_indicators)
    return macro_indicators

# 

# MAIN

In [38]:
# Main
period_length = '1M'
ticker = "SPY"
end_date = datetime.datetime(2022, 1, 1)

# Not sure why i can only do 36 months ahead
# start_date = end_date - relativedelta(months = 36)
start_date = datetime.datetime(2005, 1, 1)
start_date_with_shift = start_date - relativedelta(months = 15)

def get_data(end_date):

    date_range = pd.date_range(start=start_date, end=end_date, freq = period_length)
    macro_indicators = pd.DataFrame({'DATE': date_range})

    stock_info(macro_indicators)

    indicator_functions = [stock_info, money_supply_info, fed_interest_rate_info, cpi_info, unemployment_info]
    macro_indicators = apply_indicators_info(macro_indicators, indicator_functions)
    return macro_indicators





In [39]:
end_date = datetime.datetime(2022, 1, 1)
macro_indicators = get_data(end_date)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


# BACKTESTING

### SELL, HOLD, BUY


In [46]:
# Determine SELL, HOLD, or BUY based on future change
def buy_or_sell(training_macro_indicators):
    training_macro_indicators['future_pct_change'] = (training_macro_indicators['average_stock_price'].shift(-1) / training_macro_indicators['average_stock_price'] - 1) * 100

    conditions = [
        (training_macro_indicators['future_pct_change'] >= 0.5),
        (training_macro_indicators['future_pct_change'] < 0.5) & (training_macro_indicators['future_pct_change'] > -0.5),
        (training_macro_indicators['future_pct_change'] <= -0.5)
    ]

    choices = ['BUY', 'HOLD', 'SELL']

    training_macro_indicators['buy_or_sell'] = np.select(conditions, choices, default='HOLD')

    return training_macro_indicators

In [71]:
train_macro_indicators = macro_indicators
train_macro_indicators

,DATE,average_stock_price,average_stock_price_pct_change,lag_average_stock_price_pct_change,M1-M2,average_interest_rate,single_rate_period_change,lag_single_rate_change,average_cpi_change,single_average_cpi_change,lag_single_average_cpi_change,average_unemployment,average_unemployment_change,lag_average_unemployment_change,future_pct_change,buy_or_sell
0,2005-01-31,118.072501,-1.647602,2.338156,-1.065976,2.28,0.12,0.23,2.844874,-0.497444,-0.279304,5.3,-0.1,0.0,1.697550,BUY
1,2005-02-28,120.076841,1.697550,-1.647602,-1.447938,2.50,0.22,0.12,3.053026,0.208152,-0.497444,5.4,0.1,-0.1,-0.386135,HOLD
2,2005-03-31,119.613182,-0.386135,1.697550,-1.568602,2.63,0.13,0.22,3.206841,0.153815,0.208152,5.2,-0.2,0.1,-2.641723,SELL
3,2005-04-30,116.453333,-2.641723,-0.386135,-2.354655,2.79,0.16,0.13,3.361793,0.154952,0.153815,5.2,0.0,-0.2,1.345727,BUY
4,2005-05-31,118.020477,1.345727,-2.641723,-0.821158,3.00,0.21,0.16,2.869288,-0.492505,0.154952,5.1,-0.1,0.0,2.025825,BUY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,2021-08-31,444.520000,2.166362,2.862927,3.279482,0.09,-0.01,0.02,5.188292,-0.033214,-0.060105,5.2,-0.2,-0.5,-0.217356,HOLD
200,2021-09-30,443.553811,-0.217356,2.166362,2.929079,0.08,-0.01,-0.01,5.383630,0.195338,-0.033214,4.8,-0.4,-0.2,0.264100,HOLD
201,2021-10-31,444.725237,0.264100,-0.217356,2.683967,0.08,0.00,-0.01,6.237754,0.854124,0.195338,4.5,-0.3,-0.4,4.728870,BUY
202,2021-11-30,465.755716,4.728870,0.264100,2.509540,0.08,0.00,0.00,6.862388,0.624634,0.854124,4.2,-0.3,-0.3,0.139405,HOLD


In [76]:
full_train_macro_indicators = buy_or_sell(train_macro_indicators)
full_train_macro_indicators.dropna(subset=['future_pct_change'], inplace=True)
full_train_macro_indicators


,DATE,average_stock_price,average_stock_price_pct_change,lag_average_stock_price_pct_change,M1-M2,average_interest_rate,single_rate_period_change,lag_single_rate_change,average_cpi_change,single_average_cpi_change,lag_single_average_cpi_change,average_unemployment,average_unemployment_change,lag_average_unemployment_change,future_pct_change,buy_or_sell
0,2005-01-31,118.072501,-1.647602,2.338156,-1.065976,2.28,0.12,0.23,2.844874,-0.497444,-0.279304,5.3,-0.1,0.0,1.697550,BUY
1,2005-02-28,120.076841,1.697550,-1.647602,-1.447938,2.50,0.22,0.12,3.053026,0.208152,-0.497444,5.4,0.1,-0.1,-0.386135,HOLD
2,2005-03-31,119.613182,-0.386135,1.697550,-1.568602,2.63,0.13,0.22,3.206841,0.153815,0.208152,5.2,-0.2,0.1,-2.641723,SELL
3,2005-04-30,116.453333,-2.641723,-0.386135,-2.354655,2.79,0.16,0.13,3.361793,0.154952,0.153815,5.2,0.0,-0.2,1.345727,BUY
4,2005-05-31,118.020477,1.345727,-2.641723,-0.821158,3.00,0.21,0.16,2.869288,-0.492505,0.154952,5.1,-0.1,0.0,2.025825,BUY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2021-04-30,412.910472,5.796635,0.724004,276.553164,0.07,0.00,-0.01,4.130547,1.500028,0.937159,6.1,0.0,-0.1,0.742541,BUY
196,2021-05-31,415.976500,0.742541,5.796635,4.153750,0.06,-0.01,0.00,4.915034,0.784487,1.500028,5.8,-0.3,0.0,1.684721,BUY
197,2021-06-30,422.984544,1.684721,0.742541,3.805029,0.08,0.02,-0.01,5.281611,0.366576,0.784487,5.9,0.1,-0.3,2.862927,BUY
198,2021-07-31,435.094282,2.862927,1.684721,3.505892,0.10,0.02,0.02,5.221506,-0.060105,0.366576,5.4,-0.5,0.1,2.166362,BUY


### LGBM MODEL

In [77]:
# declare LGBM variables
var_columns = [c for c in full_train_macro_indicators if c not in['future_change_pct','DATE','buy_or_sell']]

X = full_train_macro_indicators.loc[:, var_columns]
y = full_train_macro_indicators.loc[:, 'buy_or_sell']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)

In [78]:
train_data = lightgbm.Dataset(X_train, label = y_train)
valid_data = lightgbm.Dataset(X_valid, label = y_valid)

In [79]:
# Run Light gbm model
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_valid_encoded = label_encoder.transform(y_valid)

params = {
    'objective': 'multiclass',
    'num_class': 3,
    'metric': 'multi_logloss'
}

train_data_encoded = lightgbm.Dataset(X_train, label=y_train_encoded)
valid_data_encoded = lightgbm.Dataset(X_valid, label=y_valid_encoded)

model = lightgbm.train(
    params,
    train_data_encoded,
    valid_sets=[valid_data_encoded],
    num_boost_round=5
)

predictions = model.predict(X_valid)
predicted_labels = predictions.argmax(axis=1)

try:
    # Convert predicted labels to match the format of true labels
    predicted_labels = ['BUY' if label == 0 else 'HOLD' for label in predicted_labels]

    # Evaluate the model
    accuracy = accuracy_score(y_valid, predicted_labels)
    print(f'Accuracy: {accuracy}')
except Exception as e:
    print(f'Error: {e}')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000219 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 605
[LightGBM] [Info] Number of data points in the train set: 160, number of used features: 14
[LightGBM] [Info] Start training from score -0.480054
[LightGBM] [Info] Start training from score -2.030651
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fur